In [44]:
import numpy as np

In [32]:
original_array = np.load("en_array.npy")

In [33]:
shape = original_array.shape
shape

(20, 18, 20)

In [34]:
new_shape = (2 * shape[0] - 1, 2 * shape[1] - 1, 2 * shape[2] - 1)
new_shape

(39, 35, 39)

In [35]:
new_array = np.zeros(new_shape)
new_array[::2, ::2, ::2] = original_array

In [36]:
for i in range(new_shape[0]):
    for j in range(new_shape[1]):
        for k in range(new_shape[2]):
            # the odd index is the newly added while even is the old
            r_i = i % 2
            r_j = j % 2
            r_k = k % 2
            
            # original vertices of the array (all indices are even)
            if r_i == 0 and r_j == 0 and r_k == 0:
                continue
            
            assert(new_array[i, j, k] == 0)
            # cell center (all indices are odds)
            if r_i + r_j + r_k == 3:
                new_array[i, j, k] = (new_array[i-1, j-1, k-1] + new_array[i+1, j-1, k-1] + new_array[i-1, j+1, k-1] + new_array[i+1, j+1, k-1] + 
                                      new_array[i-1, j-1, k+1] + new_array[i+1, j-1, k+1] + new_array[i-1, j+1, k+1] + new_array[i+1, j+1, k+1])/8
            # surface center (two indices are odds)
            elif r_i + r_j + r_k == 2:    
                # on the i surface
                if r_i == 0:
                    new_array[i, j, k] = (new_array[i, j-1, k-1] + new_array[i, j-1, k+1] + new_array[i, j+1, k-1] + new_array[i, j+1, k+1])/4
                # on the j surface
                elif r_j == 0:
                    new_array[i, j, k] = (new_array[i-1, j, k-1] + new_array[i-1, j, k+1] + new_array[i+1, j, k-1] + new_array[i+1, j, k+1])/4
                # on the k surface
                else:
                    new_array[i, j, k] = (new_array[i-1, j-1, k] + new_array[i-1, j+1, k] + new_array[i+1, j-1, k] + new_array[i+1, j+1, k])/4
            # edge middle
            else:
                if r_i == 1:
                    new_array[i, j, k] = (new_array[i-1, j, k] + new_array[i+1, j, k])/2
                elif r_j == 1:
                    new_array[i, j, k] = (new_array[i, j-1, k] + new_array[i, j+1, k])/2
                else:
                    new_array[i, j, k] = (new_array[i, j, k-1] + new_array[i, j, k+1])/2

In [45]:
with open('en_array_double_grids.npy', 'wb') as output_file:
    np.save(output_file, new_array)

In [46]:
new_array_reload = np.load("en_array_double_grids.npy")
new_array_reload.shape

(39, 35, 39)

In [47]:
new_array_1 = np.load("en_array_double_grids.npy")
new_array_2 = np.load("en_array_double_grids_check.npy")

In [48]:
(new_array_1 == new_array_2).all()

True

In [54]:
import pandas as pd
from itertools import product

x, y, z = new_array_reload.shape
x_, y_, z_ = zip(*product(range(x), range(y), range(z)))
df = pd.DataFrame(new_array_reload.flatten(), columns=["value"]).assign(x=x_, y=y_, z=z_)

In [56]:
df["x"] = df["x"]/2.0
df["y"] = df["y"]/2.0
df["z"] = df["z"]/2.0

In [57]:
df

,value,x,y,z
0,-214.664780,0.0,0.0,0.0
1,-188.833805,0.0,0.0,0.5
2,-163.002830,0.0,0.0,1.0
3,-16.615820,0.0,0.0,1.5
4,129.771190,0.0,0.0,2.0
...,...,...,...,...
53230,-223.617880,19.0,17.0,17.0
53231,-223.618140,19.0,17.0,17.5
53232,-223.618400,19.0,17.0,18.0
53233,-223.620515,19.0,17.0,18.5


In [58]:
df.to_csv("en_array_double_grids.csv", index=False)